In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os.path
import time
import tensorflow as tf

In [2]:
flags = tf.app.flags
FLAGS = flags.FLAGS
flags.DEFINE_float('learning_rate', 0.01, 'Initial learning rate.')
flags.DEFINE_integer('num_epochs', 2, 'Number of epochs to run trainer.')
flags.DEFINE_integer('batch_size', 128, 'Batch size.')
flags.DEFINE_string('train_dir', '/data', 'Directory with the training data.')

In [7]:
TRAIN_FILE = "tf-records.data"
VAL_FILE = ""

In [8]:
def extract_features(serialized_example):
    """
    Extracts a `dict` of named features from the serialized `tf.train.Example`
    """
    return tf.parse_single_example(
        serialized=serialized_example,
        features={
            'label': tf.FixedLenFeature([99], dtype=tf.int64),
            'input': tf.FixedLenFeature([20,111], dtype=tf.int64),
        }
    )

def deserialize_example(serialized_example):
    """
    Converts a serialized `tf.train.Example` to FP32 Tensors
    """
    features = extract_features(serialized_example)
    input = tf.cast(features['input'], tf.float32)
    label = tf.cast(features['label'], tf.float32)
    return input, label

def read_and_decode(filename_queue):
    """
    Read and Deserialize a single `tf.train.Example` from a TFRecord file.
    """
    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_queue)
    return deserialize_example(serialized_example)

In [13]:
def inputs(batch_size=None, num_epochs=None, train=True):
    """
    Read in shuffled `inputs` and `labels` from either train or val files.
    
    Returns:
        `inputs` : [batch_size, 20, 111]
        `labels` : [batch_size, 99]
    """
    batch_size = batch_size or 128
    num_epochs = num_epochs or 1
    
    filename = os.path.join(FLAGS.train_dir, TRAIN_FILE if train else VALIDATION_FILE)
    print(filename)
    
    with tf.name_scope('input'):
        filename_queue = tf.train.string_input_producer([filename], num_epochs=num_epochs)
        input, label = read_and_decode(filename_queue)
        inputs, labels = tf.train.shuffle_batch(
            [input, label], batch_size=batch_size, num_threads=2,
            capacity=1000 + 3 * batch_size,
            # Ensures a minimum amount of shuffling of examples.
            min_after_dequeue=1000
        )
        return inputs, labels

## Test Reading Input

The following code block reads the first batch of input data and print out the details.

In [15]:
with tf.Session() as sess:
    input, label = inputs(batch_size=4, num_epochs=1)
    #filename = os.path.join(FLAGS.train_dir, TRAIN_FILE if True else VALIDATION_FILE)
    #filename_queue = tf.train.string_input_producer([filename], num_epochs=1)
    #input, label = read_and_decode(filename_queue)
    init_op = tf.initialize_all_variables()
    local_op = tf.initialize_local_variables()
    sess.run(init_op)
    sess.run(local_op)
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    
    try:
        example = sess.run([input, label])
        print(input)
        print(label)
    except OutOfRangeError, e:
        print("out of range")
        coord.request_stop(e)
    finally:
        coord.request_stop()
        coord.join(threads)

/data/tf-records.data
Tensor("input_2/shuffle_batch:0", shape=(4, 20, 111), dtype=float32)
Tensor("input_2/shuffle_batch:1", shape=(4, 99), dtype=float32)
